#a)
Cargue la base de datos, ¿hay alguna columna que no sea útil para el
análisis?. Revise si existen casos faltantes en la base de datos, según la
descripción de las variables, ¿qué pudiera significar un NA? Tome
decisiones de qué hacer si existen casos faltantes. Si desea imputar los
valores nulos de alguna variable por un valor puede utilizar
data["columna"].fillna('valor', inplace = True).

In [1]:
import pandas as pd

# Cargar los datos con el delimitador correcto
df = pd.read_csv('house_price.csv', sep=';')

# Revisar las primeras filas y la información general del dataframe para identificar columnas no útiles y valores faltantes
#df.head(), df.info()
df.shape

(1460, 21)

In [2]:
# Verificar la naturaleza de los valores faltantes en las columnas 'LotFrontage' y 'MasVnrArea'
# ¿Son NaN, nulos o ceros?

valores_nulos_lotfrontage = df['LotFrontage'].isnull().sum()
valores_nulos_masvnrarea = df['MasVnrArea'].isnull().sum()

valores_ceros_lotfrontage = (df['LotFrontage'] == 0).sum()
valores_ceros_masvnrarea = (df['MasVnrArea'] == 0).sum()

valores_nulos_lotfrontage, valores_nulos_masvnrarea, valores_ceros_lotfrontage, valores_ceros_masvnrarea


(259, 8, 0, 861)

In [3]:
# Imputar el valor medio en las columnas con valores NaN
df['LotFrontage'].fillna(df['LotFrontage'].mean(), inplace=True)
df['MasVnrArea'].fillna(df['MasVnrArea'].mean(), inplace=True)

# Verificar si aún hay valores NaN en las columnas modificadas
nan_count_after_imputation = df[['LotFrontage', 'MasVnrArea']].isna().sum()

nan_count_after_imputation


LotFrontage    0
MasVnrArea     0
dtype: int64

b) La variable a modelar corresponde al precio de venta de la casa en dólares. En base a los modelos que hemos aprendido en clases, ¿qué modelo(s) podría(n) ser de utilidad para predecir el precio de venta de casas mediante las variables predictoras?

Respuesta: se podrian utilizar modelo de Regresión lineal, regresión polinomial, regresión logística, árboles de decisión o random forest.

c) Vamos a implementar un modelo de regresión lineal para modelar el precio de venta de casas. Considerando que en la base de datos tenemos varias variables que dependen de otras, ¿cuál pudiera ser el problema que surgiría en este contexto? ¿por qué esto es un problema?
Discuta.

Respuesta: el problema que surge es la multicolinealidad, ya que las variables independientes estan correlacionadas entre si. Para solucionar este problema se puede utilizar la técnica de reducción de dimensionalidad PCA.

#d)
Realice un análisis de la correlación entre las variables. Comente,
¿existen variables muy correlacionadas entre sí? ¿Por qué cree que
ocurre?

In [4]:
# Calcular la matriz de correlación
correlation_matrix = df.corr()

# Mostrar la matriz de correlación
correlation_matrix


,Id,LotFrontage,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice
Id,1.000000,-0.009601,-0.033226,-0.050199,-0.005024,-0.005968,-0.007940,-0.015415,0.010496,0.005590,...,0.008273,0.017634,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,-0.021917
LotFrontage,-0.009601,1.000000,0.306795,0.179283,0.215828,0.043340,0.122156,0.363358,0.414266,0.072483,...,0.368392,0.323663,0.077106,0.137454,0.009790,0.062335,0.037684,0.180868,0.001168,0.334901
LotArea,-0.033226,0.306795,1.000000,0.103960,0.214103,0.111170,-0.002618,0.260833,0.299475,0.050986,...,0.263116,0.180403,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.263843
MasVnrArea,-0.050199,0.179283,0.103960,1.000000,0.263582,-0.072302,0.114184,0.362452,0.342160,0.174019,...,0.389893,0.372567,0.159349,0.124965,-0.109849,0.018795,0.061453,0.011723,-0.029815,0.475241
BsmtFinSF1,-0.005024,0.215828,0.214103,0.263582,1.000000,-0.050117,-0.495251,0.522396,0.445863,-0.137079,...,0.208171,0.296970,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571,0.386420
BsmtFinSF2,-0.005968,0.043340,0.111170,-0.072302,-0.050117,1.000000,-0.209294,0.104810,0.097117,-0.099260,...,-0.009640,-0.018227,0.067898,0.003093,0.036543,-0.029993,0.088871,0.041709,0.004940,-0.011378
BsmtUnfSF,-0.007940,0.122156,-0.002618,0.114184,-0.495251,-0.209294,1.000000,0.415360,0.317987,0.004469,...,0.240257,0.183303,-0.005316,0.129005,-0.002538,0.020764,-0.012579,-0.035092,-0.023837,0.214479
TotalBsmtSF,-0.015415,0.363358,0.260833,0.362452,0.522396,0.104810,0.415360,1.000000,0.819530,-0.174512,...,0.454868,0.486665,0.232019,0.247264,-0.095478,0.037384,0.084489,0.126053,-0.018479,0.613581
1stFlrSF,0.010496,0.414266,0.299475,0.342160,0.445863,0.097117,0.317987,0.819530,1.000000,-0.202646,...,0.566024,0.489782,0.235459,0.211671,-0.065292,0.056104,0.088758,0.131525,-0.021096,0.605852
2ndFlrSF,0.005590,0.072483,0.050986,0.174019,-0.137079,-0.099260,0.004469,-0.174512,-0.202646,1.000000,...,0.687501,0.138347,0.092165,0.208026,0.061989,-0.024358,0.040606,0.081487,0.016197,0.319334


#f)
Obtenga las componentes principales utilizando las variables
estandarizadas, si desea explicar un 80% de la variabilidad, ¿cuántas
componentes debería elegir? Argumente.

In [ ]:
from sklearn.impute import SimpleImputer
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Preparación de los datos
X = df.drop(columns=['SalePrice', 'Id'])  # Eliminamos la columna objetivo y el Id
y = df['SalePrice']

# Imputar valores medios a los NaN en todas las columnas numéricas
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Dividir los datos en conjuntos de entrenamiento y prueba nuevamente
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LinearRegression()

# Calcular el tiempo de procesamiento
start = datetime.now()
model.fit(X_train, y_train)
end = datetime.now()

# Predicción y cálculo del error cuadrático medio
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)

# Tiempo de procesamiento
processing_time = end - start

print(f'Tiempo de procesamiento: {processing_time}, RMSE: ${np.sqrt(mse)}')


Tiempo de procesamiento: 0:00:00.101585, RMSE: $43058.010783017206


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Inicializar PCA
pca = PCA()

# Realizar imputación para todas las columnas, utilizando el valor medio para cada una
X_imputed = X.fillna(X.mean())

# Estandarizar las características después de la imputación
X_scaled_imputed = StandardScaler().fit_transform(X_imputed)

# Ajustar PCA
pca.fit(X_scaled_imputed)

# Calcular la suma acumulada de la varianza explicada después de la imputación
varianza_acumulada_imputed = pca.explained_variance_ratio_.cumsum()

# Encontrar el número mínimo de componentes para explicar al menos el 80% de la variabilidad después de la imputación
num_componentes_imputed = (varianza_acumulada_imputed >= 0.8).argmax() + 1

num_componentes_imputed, varianza_acumulada_imputed


(11,
 array([0.20974689, 0.30429316, 0.38433835, 0.45265864, 0.51107426,
        0.56664554, 0.61963792, 0.67188937, 0.72186245, 0.76904562,
        0.8141376 , 0.85895596, 0.89688651, 0.93017357, 0.95948463,
        0.98736275, 1.        , 1.        , 1.        ]))